# Synthetic model of a ten polygonal close to a cylinder with magnetization

In [1]:
import matplotlib

In [2]:
matplotlib.use('TkAgg', force=True)

In [3]:
import numpy as np
import cPickle as pickle

from copy import deepcopy
from fatiando import utils, mesher
from fatiando.mesher import PolygonalPrism
from fatiando.vis import mpl, myv
from matplotlib import colors, ticker, cm
from IPython.display import Image as img
import mag_polyprism_functions as mfun

/home/vital/anaconda2/envs/radial-mag/lib/python2.7/site-packages/fatiando/vis/mpl.py:76: UserWarning: This module will be removed in v0.6. We recommend the use of matplotlib.pyplot module directly. Some of the fatiando specific functions will remain.
  "specific functions will remain.")
/home/vital/anaconda2/envs/radial-mag/lib/python2.7/site-packages/fatiando/vis/myv.py:51: UserWarning: This module will be removed in v0.7.
  warnings.warn("This module will be removed in v0.7.")


In [4]:
handfree_model = dict()

In [5]:
handfree_model['z0'] = 0.    # depth of the top the shallowest prism
handfree_model['dz'] = 3000.    # thickness of each prism

handfree_model['inc'] = -60.  # inclination
handfree_model['dec'] = 50.   # declination
handfree_model['intensity'] = 3. # intensity

handfree_model['props']={'magnetization': utils.ang2vec(
    handfree_model['intensity'], handfree_model['inc'], handfree_model['dec'])} # physical property: magnetization

In [6]:
bounds = [-10000, 10000, -10000, 10000, 0, 5000]
area = bounds[:4]
depths = [handfree_model['z0'], handfree_model['dz']]
model = []
for i in range(1, len(depths)):
    axes = mpl.figure().gca()
    mpl.axis('scaled')
    for p in model:
        mpl.polygon(p, '.-k', xy2ne=True)
    model.append(
        mesher.PolygonalPrism(
            mpl.draw_polygon(area, axes, xy2ne=True),
            depths[i - 1], depths[i], handfree_model['props']))

In [7]:
handfree_model['prisms'] = model

In [8]:
handfree_model['props'].type

AttributeError: 'dict' object has no attribute 'type'

In [15]:
M = model[0].x.size
L = len(model)
P = L*(M+2)
m = np.zeros(P)
for i, mod in enumerate(model):
        x = np.asanyarray(mod.x)
        y = np.asanyarray(mod.y)
        n = len(x)
        shift_up = np.arange(-n+1, 1)
        shift_down = np.arange(-1, n-1)
        area = -(x * (y.take(shift_up) - y.take(shift_down))).sum() / 2.0
        cx = ((x + x.take(shift_up))*x*(y.take(shift_up) - y.take(shift_down))).sum()
        cx /= 6.*area
        cy = (y + y.take(shift_up)*x*(y.take(shift_up) - y.take(shift_down))).sum()
        cy /= 6.*area
        m[i*(M+2):i*(M+2)+M] = np.sqrt((x - cx)*(x - cx) + (y - cy)*(y - cy))
        m[i*M+M] = cx
        m[i*M+M+1] = cy

In [16]:
print m

[  890.96886745  5744.6616419   4603.9154344   3670.92833157  4252.38450167
   573.61375205  -209.50794662]


In [17]:
handfree_model['M'] = handfree_model['prisms'][0].x.size # number of vertices per prism
handfree_model['L'] = len(handfree_model['prisms']) # number of prisms
handfree_model['P'] = handfree_model['L']*(handfree_model['M']+2) # number of parameters

In [18]:
mi = mfun.param2polyprism(m, handfree_model['M'], handfree_model['L'], handfree_model['z0'], handfree_model['dz'], handfree_model['props'])

In [19]:
# Show the model
myv.figure()
myv.polyprisms(handfree_model['prisms'], 'magnetization', opacity=1., style='wireframe')
myv.polyprisms(mi, 'magnetization', opacity=.5, linewidth=0.5)
myv.axes(myv.outline(bounds), ranges=[i * 0.001 for i in bounds])
myv.show()

In [20]:
# Cartesian coordinates of the origin of each prism
x0 = np.zeros(cylinder_model['L']) 
y0 = np.zeros(cylinder_model['L'])

l = []   # list of parameters of the prisms

for i in range(cylinder_model['L']):
    l.append([r, x0[i]+100., y0[i]+100., cylinder_model['z0'] + cylinder_model['dz']*i,\
              cylinder_model['z0'] + cylinder_model['dz']*(i + 1), cylinder_model['props']])
    
cylinder_model['prisms'] = mfun.pol2cart(l, cylinder_model['M'], cylinder_model['L'])
cylinder_model['param_vec'] = mfun.param_vec(l, cylinder_model['M'], cylinder_model['L'])

NameError: name 'cylinder_model' is not defined

In [ ]:
file_name = '../data/handfree_model.pickle'
with open(file_name, 'w') as f:
    pickle.dump(handfree_model, f)